<p>https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=11128</p>
<p>https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=[TaxId]</p>

In [ ]:
import re, os, json

import numpy as np

from Bio import Entrez

Entrez.email = "VIMVer@univ-amu.fr"

testydata: str = "./../../../testymolo/media/data/"
tables_csv: str = "./../../../testymolo/media/tables_csv/"

merge_json: str = "./../../../testymolo/media/data/"

In [ ]:
### caution when not all rows same nbr of col
def custom_csv_parser_to_list(infilepath:str):
    with open(infilepath ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            values.append(val)
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                values.append(val)

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        print("nbr of col", set([len(row) for row in rows]))
        print("nbr of rows", len(rows))
        return rows

In [ ]:
Organism_csv:list = custom_csv_parser_to_list(os.path.join(tables_csv, 'Organism.csv'))
Organism_h:list = ["Tax_id", "Name", "Categorie", "Classe", "Ordre", "Fam", "SsFam", "Genre", "Note_org"]

CAZy_DB_csv:list = custom_csv_parser_to_list(os.path.join(tables_csv, 'CAZy_DB.csv'))
CAZy_DB_h:list = ["DB_ac", "Protein", "DB_nom", "Organism", "abr", "Tax_id", "EC", "_3D_status",
                   "Length", "Sequence", "DB_note", "Created", "Modified", "PP_status", "Lib_sort"]


In [ ]:
def read_merge_data(infilepath:str):
    with open(infilepath, 'r') as handle :
        try:
            data = json.loads(handle.read())
        except:
            data = {}
            print("ERROR")
    return data

In [ ]:
Organism_json:list = read_merge_data(os.path.join(merge_json, 'Organism.temp.json'))

In [ ]:
for item in Organism_json:
    for key, value in item.items():
        print(key, value)
    break

In [ ]:
TaxId:int = 11120

print("Organism_csv")
for item in Organism_csv:
    if str(TaxId) == str(item[0]):
        print(item)
print("CAZy_DB_csv")
for item in CAZy_DB_csv:
    if str(TaxId) == str(item[5]):
        print(item)


In [ ]:
import subprocess as sh

#var = sh.run(['taxonkit', 'list', '--ids', '11120', '--indent', ''])
#  taxonkit list --show-rank --show-name --indent "    " --ids 11120 -J
var = sh.run(["taxonkit","list", "--show-rank", "--show-name", "--indent", "    ", "--ids", "11128", "-J"], stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print(var.stdout)

In [ ]:

import json
var_json = json.loads(var.stdout)

def json2html(data:dict) -> str:
    #recursive
    txt:str = ""
    for line, subdata in data.items():
        line:str = line.strip()
        txt += "<li>"+line+"</li>"
        
        if len(subdata) > 0: #sub_clade
            txt += "<ul>" + json2html(subdata) + "</ul>"
    return txt

var_html = "<div class='taxonkit'><ul>"+json2html(var_json)+"</ul></div>"
print(var_html)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(var_html))

In [ ]:

display(HTML("<div><ul><li>species</li><ul><li>sub-species</li><li>sub-species</li></ul><li>species</li><li>species</li></ul></div>"))

In [ ]:
command = sh.run(['echo', 'Za Worudo !'], stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print(command.stdout)

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))


# NCBI genbank record

<ul>genome
    <li>ORF</li>
    <li>cds</li></ul>
<ul>proteome
    <li>protein</li></ul>

<ul>
<li>scr</li>
<li>view</li>
<li>url</li>
<li>js</li>
<li>html</li></ul>

In [ ]:
#does it work every time

Protein_json:list = read_merge_data(os.path.join(merge_json, 'Protein.temp.json'))

In [ ]:
for org in Organism_json:
    taxid = str(org['id'])
    proteins:list = [ prot for prot in Protein_json if str(prot['organism']) == taxid ]
    for prot in proteins:
        terms:str = f"{org['name']}[Organism] AND {prot['name']} AND refseq[filter]"
        print(terms)
        with Entrez.esearch(db='nucleotide', term=terms, idtype='acc') as handle :
            record = Entrez.read(handle)
            print(record['IdList'])

In [ ]:
# get complementary info on each

for accNumber in record['IdList']:
    print(accNumber)
    genbank = Entrez.efetch(db='nucleotide', id=accNumber, rettype='gb', retmode='text')
    print(genbank.readlines()[0].strip())


In [23]:
# run blast
#blastp -query ./protein_test.fasta -db refseq_protein -max_target_seqs 5 -remote -outfmt 5 > var.blastp.html

import subprocess as sh

command_line = ['blastp', '-query', './protein_test.fasta', '-db', 'refseq_protein', '-max_target_seqs', '10', '-remote', '-outfmt', '5', '-out', './var.temp.blastp.xml']
command_line = sh.run(command_line, stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print('uwu', command_line.stdout)


uwu 


In [16]:
import Bio.Blast.NCBIXML as BlastXML

with open("./.var.temp.blastp.xml") as blastp_result:
    result = BlastXML.parse(blastp_result)
    BlastXML.parse()

    for record in result:
        print(record)

FileNotFoundError: [Errno 2] No such file or directory: './.var.temp.blastp.xml'